In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.19.0


In [2]:
names= set()
txt = 'family.txt'
with open(txt,'r') as file:
    name = file.readlines()
for name in name[6:]:
    names.add(name.split()[0])

In [3]:
import csv
txt = 'train.txt'
with open(txt, 'r') as file:
    data = file.readlines()
validation_len = len(data)/5
validation_set = data[0:validation_len]
train_set = data[validation_len:]

In [4]:
def extractShape(word,name):
    shape=name
    if word[0].isupper():
        shape += 'Xx'
    if '-' in word:
        index = word.index('-')
        shape += '-'
        if index+1<len(word) and word[index+1].isupper():
            shape += 'X'
        else: shape += 'x'
        if index+2<len(word):
            shape += 'x'
    return shape

In [5]:
def extractSufPre(word,maxlen):
    features = []
    length = len(word)
    maxlen = max(length,maxlen)
    for i in range(1,maxlen+1):
        features.append(word[:i])
        features.append(word[-i:])
    return features

In [6]:
def extractHypen(word,name):
    features = []
    if '-' in word:
        features.append(name+'hyphen')
        index = word.index('-')
        features.append(name+word[:index])
        features.append(name+word[index:])
    return features

In [7]:
def numberGen(word):
    result=''
    for char in word:
        if char.isdigit():
            result +='D'
        else:
            result +=char
    return result

In [8]:
def isName(word):
    if word in names:
        return 'True'
    else: return 'False'

In [16]:
# Extract features from words. 
def word2features(word_line,pos_line,i):
    word = word_line[i]
    postag = pos_line[i]
    features = [
        'word.lower=' + word.lower(),
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
        'shape='+extractShape(word,'word'),
        'numGen'+numberGen(word),
        isName(word)
    ]
    features.extend(extractSufPre(word,6))
    features.extend(extractHypen(word,'word'))
    if i > 0:
        word1 = word_line[i-1]
        postag1 = pos_line[i-1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
            '-1:shape='+extractShape(word1,'-1word'),
            'numGen'+numberGen(word1),
            '-1:Cap%s'%word1[0].isupper(),
            isName(word)

        ])
        features.extend(extractHypen(word1,'-1word'))

    else:
        features.append('BOS')
        
    if i < len(word_line)-1:
        word1 = word_line[i+1]
        postag1 = pos_line[i+1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
            '+1:shape='+extractShape(word1,'+1word'),
            'numGen'+numberGen(word1),
            '+1:Cap%s' %word1[0].isupper() ,
            isName(word)
        ])
        features.extend(extractHypen(word1,'+1word'))
    else:
        features.append('EOS')
                
    return features   

In [17]:
# contruct train and test set for trainer
def data2features(data):
    X = []
    Y = []
    for i in range(0,len(data),3):
        word_line = data[i].split()
        pos_line = data[i+1].split()
        NE_line = data[i+2].split()

        feature = [word2features(word_line,pos_line,i) for i in range(len(word_line))]
        label = NE_line

        X.append(feature)
        Y.append(label)
    return X,Y

In [18]:
X_train,y_train = data2features(train_set)
X_test,y_test = data2features(validation_set)

trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [ ]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
        digits=4
    )

In [19]:
for c1 in [0.05]:
    for c2 in [0.05]:
        for max_iterations in [35]:
            
            trainer.set_params({
                'c1': c1,   # coefficient for L1 penalty
                'c2': c2,  # coefficient for L2 penalty
                'max_iterations': max_iterations,  # stop earlier


                # include transitions that are possible, but not observed
                'feature.possible_transitions': True,

            })
            trainer.train('NER')
            tagger = pycrfsuite.Tagger()
            tagger.open('NER')
            y_pred = [tagger.tag(xseq) for xseq in X_test]
            print(bio_classification_report(y_test, y_pred))

c10.05 c20.05 maxit35
             precision    recall  f1-score   support

      B-LOC     0.8874    0.9114    0.8993      1332
      I-LOC     0.9079    0.8543    0.8803       254
     B-MISC     0.9179    0.8798    0.8985       699
     I-MISC     0.8543    0.8293    0.8416       205
      B-ORG     0.8880    0.8126    0.8486      1297
      I-ORG     0.8436    0.7893    0.8155       745
      B-PER     0.8645    0.8380    0.8511      1241
      I-PER     0.8936    0.9535    0.9226       775

avg / total     0.8819    0.8609    0.8707      6548



In [7]:
# helper function to output the NE results
def tagging(begin_index,end_index,tag):
    global PER,LOC,ORG,MISC
    if tag == 'PER':
         PER =  PER+str(begin_index)+'-'+str(end_index)+' '
    if tag == 'LOC':
        LOC =  LOC+str(begin_index)+'-'+str(end_index)+' '
    if tag == 'ORG':
        ORG =  ORG+str(begin_index)+'-'+str(end_index)+' '
    if tag == 'MISC':
        MISC = MISC+str(begin_index)+'-'+str(end_index)+' '

In [8]:
# helper function2 to output the NE results
def tagging2(tags,num_line):
    begin = False
    for i in range(len(tags)):
        if tags[i][0]=='B':
            begin_index = num_line[i]
            if begin == True:
                tag = tags[i-1][2:]
                end_index = num_line[i-1]
                tagging(begin_index,end_index,tag)
            begin = True
            
        if tags[i][0]=='O' and begin==True:
                tag = tags[i-1][2:]
                end_index = num_line[i-1]
                tagging(begin_index,end_index,tag)
                begin = False
                
        if i == len(tags) and begin==True:
            tag = tags[i][2:]
            end_index = num_line[i]
            tagging(begin_index,end_index,tag)

In [101]:
txt = 'train.txt'
with open(txt, 'r') as file:
    data = file.readlines()

X_train,y_train = data2features(data)
trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)
    
trainer.set_params({
    'c1': 0.05,   # coefficient for L1 penalty
    'c2': 0.05,  # coefficient for L2 penalty
    'max_iterations': 35,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

trainer.train('NER')
tagger = pycrfsuite.Tagger()
tagger.open('NER')

In [102]:
headers = ['Type','Prediction']
rows = []
PER = ''
LOC = ''
ORG = ''
MISC = ''

txt = 'test.txt'
with open(txt, 'r') as file:
    data = file.readlines()

for i in range(0,len(data),3):
        word_line = data[i].split()
        pos_line = data[i+1].split()
        num_line = data[i+2].split()
    
        feature = [word2features(word_line,pos_line,i) for i in range(len(word_line))]
        tags = tagger.tag(feature)
        tagging2(tags,num_line)
        


rows.append(('PER',PER))
rows.append(('LOC',LOC))
rows.append(('ORG',ORG))
rows.append(('MISC',MISC))
               
            
with open('CRF_2.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)  

In [5]:
def stanfordNE2BIO(tags):
    result=[]
    prev_tag = "O"
    for word,tag in tags:
        if tag == "O": #O
            result.append(tag)
            prev_tag = tag
            continue
        if tag != "O" and prev_tag == "O": # Begin NE
            real_tag = tag[0:3]
            if real_tag=='MIS':real_tag+='C'
            result.append("B-"+real_tag)
            prev_tag = tag
        elif prev_tag != "O" and prev_tag == tag: # Inside NE
            real_tag = tag[0:3]
            if real_tag=='MIS':real_tag+='C'
            result.append("I-"+real_tag)
            prev_tag = tag
        elif prev_tag != "O" and prev_tag != tag: # Adjacent NE
            real_tag = tag[0:3]
            if real_tag=='MIS':real_tag+='C'
            result.append("I-"+real_tag)
            prev_tag = tag

    return result

In [3]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('stanford-ner/classifiers/english.conll.4class.distsim.crf.ser.gz',
   'stanford-ner/stanford-ner-3.8.0.jar',
   encoding='utf-8')

text = 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

print(classified_text)

[(u'While', u'O'), (u'in', u'O'), (u'France', u'LOCATION'), (u',', u'O'), (u'Christine', u'PERSON'), (u'Lagarde', u'PERSON'), (u'discussed', u'O'), (u'short-term', u'O'), (u'stimulus', u'O'), (u'efforts', u'O'), (u'in', u'O'), (u'a', u'O'), (u'recent', u'O'), (u'interview', u'O'), (u'with', u'O'), (u'the', u'O'), (u'Wall', u'ORGANIZATION'), (u'Street', u'ORGANIZATION'), (u'Journal', u'ORGANIZATION'), (u'.', u'O')]


In [9]:
headers = ['Type','Prediction']
rows = []
PER = ''
LOC = ''
ORG = ''
MISC = ''

txt = 'test.txt'
with open(txt, 'r') as file:
    data = file.readlines()

for i in range(0,len(data),3):
        word_line = data[i].split()
        num_line = data[i+2].split()
    
        tags = st.tag(word_line)
        tags = stanfordNE2BIO(tags)
        tagging2(tags,num_line)
        


rows.append(('PER',PER))
rows.append(('LOC',LOC))
rows.append(('ORG',ORG))
rows.append(('MISC',MISC))
               
            
with open('CRF_2.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)  